In [1]:
import pandas as pd 
import numpy as np 

In [2]:
pd_data = pd.read_csv(
    './vegeterianl_dataset.csv',  delimiter=r"\s+")
pd_data.drop("ID",axis=1)
pd_data

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
0,1,False,steep,3900,chapparal
1,2,True,moderate,300,riparian
2,3,True,steep,1500,riparian
3,4,False,steep,1200,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer
6,7,True,steep,3000,chapparal


In [3]:
pd_data.sort_values("ELEVATION")

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
1,2,True,moderate,300,riparian
3,4,False,steep,1200,chapparal
2,3,True,steep,1500,riparian
6,7,True,steep,3000,chapparal
0,1,False,steep,3900,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer


In [4]:
def get_info(df):
    riparian = df.loc[df["VEGETATION"]=="riparian"]
    chapparal = df.loc[df["VEGETATION"]=="chapparal"]
    conifer = df.loc[df["VEGETATION"]=="conifer"]

    x = np.array([len(riparian)/len(df),len(chapparal)/len(df), 
                  len(conifer)/len(df)])
    y = np.log2(x[x!=0])
    

    info_all = - sum(x[x!=0] * y)
    return info_all

In [5]:
get_info(pd_data)

1.5566567074628228

In [10]:
def get_attribute_info(df, attribute_name, continuous_value=0):
    get_infos = []
    if continuous_value == 0:
        attribute_values = pd_data[attribute_name].unique()
        for value in attribute_values:
            split_df = pd_data.loc[pd_data[attribute_name] == value]
            get_infos.append((len(split_df) / len(df)) * get_info(split_df))
    else:
        split_df_1 = pd_data.loc[pd_data[attribute_name] >= continuous_value]
        split_df_2 = pd_data.loc[pd_data[attribute_name] < continuous_value]

        get_infos.append((len(split_df_1) / len(df)) * get_info(split_df_1))                     
        get_infos.append((len(split_df_2) / len(df)) * get_info(split_df_2))                     
    return sum(get_infos)

In [11]:
def get_continuos_attribute_info(df, attribute_name):
    result = pd_data.sort_values(attribute_name)
    prior = 0 
    split_index = []
    sorted_key = {}
    sorted_index = {}
    for index, value in enumerate(result.index):
        if prior != result["VEGETATION"][value]:
            split_index.append(value)
            prior = result["VEGETATION"][value]
        sorted_index[value] = index
        sorted_key[index] = value
    split_value = []
    for v in split_index[1:]:
        current_value = result[attribute_name][v]
        prior_value_index = sorted_key[sorted_index[v] - 1]
        prior_value = result[attribute_name][prior_value_index]
        
        split_value_average = current_value + prior_value
        split_value.append(split_value_average/2)
    result = {}
    for value in split_value:
        result[value] = (get_attribute_info(df, attribute_name, value))
    return result

In [12]:
get_continuos_attribute_info(pd_data, "ELEVATION")

{750.0: 1.250698214594781,
 1350.0: 1.3728057820624016,
 2250.0: 0.9649839288804954,
 4175.0: 0.6935361388961918}

In [222]:
for key, value in get_continuos_attribute_info(pd_data, "ELEVATION").items():
    print(key, ":", get_info(pd_data) - value)

750.0 : 0.305958492868
1350.0 : 0.1838509254
2250.0 : 0.591672778582
4175.0 : 0.863120568567


In [36]:
get_info(pd_data) - get_attribute_info(pd_data, "STREAM")

0.30595849286804166

In [37]:
get_info(pd_data) - get_attribute_info(pd_data, "SLOPE")

0.57740628285234519